# Replicate Paper


In [ ]:
%pip install torch -q

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

## Generate XORshift sequence

In [ ]:
def xorshift128():
    '''xorshift
    https://en.wikipedia.org/wiki/Xorshift
    '''

    x = 123456789
    y = 362436069
    z = 521288629
    w = 88675123

    def _random():
        nonlocal x, y, z, w
        t = x ^ ((x << 11) & 0xFFFFFFFF)  # 32bit
        x, y, z = y, z, w
        w = (w ^ (w >> 19)) ^ (t ^ (t >> 8))
        return w

    return _random



x = open('xorshift128.txt','w')
r = xorshift128()

for i in range(4000000):
    _ = x.write(str(r())+'\n')

In [ ]:
IMPORT_COUNT = 1990000
TEST_COUNT = 10000

In [ ]:
myrand=71926
np.random.seed(myrand)
torch.manual_seed(myrand)
print("Random seed is:",myrand)

In [ ]:
PREVIOUS_TIMESTEP_COUNT = 4
TOTAL_DATA_NUM = IMPORT_COUNT-PREVIOUS_TIMESTEP_COUNT

In [ ]:
# convert the sequence of generated numbers to 4 inputs and one output
def strided(a, L):
	shp = a.shape
	s  = a.strides
	nd0 = shp[0]-L+1
	shp_in = (nd0,L)+shp[1:]
	strd_in = (s[0],) + s
	return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

In [ ]:
RNG_OUTPUT_FILENAME="xorshift128.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter='\n',dtype='uint64')[:IMPORT_COUNT]

In [ ]:
# calculates how many bits are in the output.
BIT_WIDTH = np.ceil(np.log2(np.amax(df))).astype(int)

In [ ]:
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)
df_as_frames = strided(df_as_bits, PREVIOUS_TIMESTEP_COUNT+1)

In [ ]:
indicies = np.arange(TOTAL_DATA_NUM,dtype='uint64')
np.random.shuffle(indicies)
df_as_frames=df_as_frames[indicies]

In [ ]:
# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])

In [ ]:
# Convert the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [ ]:
X_train.shape

In [ ]:
X_test[:10]

In [ ]:
class XORShift128Model(nn.Module):
    def __init__(self):
        super(XORShift128Model, self).__init__()
        self.fc1 = nn.Linear(128, 1024)  # First dense layer
        self.fc2 = nn.Linear(1024, 32)   # Second dense layer, output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = XORShift128Model()

# Print the model structure
print(model)

In [ ]:
# Define a custom dataset class
class XORDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_data = XORDataset(X_train,y_train)
test_data = XORDataset(X_test, y_test)

In [ ]:
batch_size = 512  # Adjust this size to your needs
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_data, batch_size=512, shuffle=False)

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training on device: {device}.")

## Set up loss

In [ ]:
criterion = nn.BCEWithLogitsLoss()
## These are taking from the source code
hp = {
    'learning_rate': 0.0003826518055596723,
    'epsilon': 6.396190758353597e-07,
    'beta_1': 0.8494228963384591,
    'beta_2': 0.8782732182358544
}

# Configure the optimizer with these hyperparameters
optimizer = torch.optim.NAdam(model.parameters(), 
                              lr=hp['learning_rate'], 
                              betas=(hp['beta_1'], hp['beta_2']),
                              eps=hp['epsilon'])
model = model.to(device)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    for inputs, targets in train_data_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)  
        loss = criterion(outputs, targets)  
        loss.backward()
        optimizer.step()

    # Epoch end
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
torch.save(model.state_dict(), "pytorch_model.pth")